In [ ]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
# Import CV
f=open('textfile.txt','r', errors = 'ignore')
text=f.read()

In [ ]:
def nlp(x):
    word_sent = word_tokenize(x.lower().replace("\n",""))
    _stopwords = set(stopwords.words('english') + list(punctuation)+list("●")+list('–')+list('’'))
    word_sent=[word for word in word_sent if word not in _stopwords]
    lemmatizer = WordNetLemmatizer()
    NLP_Processed_CV = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(word_sent))]
#     return " ".join(NLP_Processed_CV)
    return NLP_Processed_CV

In [ ]:
def get_key_word(x):
    finder = BigramCollocationFinder.from_words(x)
    lst = []
    Key_word_from_CV = []
    for i in sorted(finder.ngram_fd.items()):
    # if a double word appears more than once, then print it out.
        if i[1] > 1:
            print(i)
            lst.append(i[0])
        else:
            pass

    # print("************************")
    for j in lst:
    #     print(" ".join(j))
        Key_word_from_CV.append(" ".join(j))
    return Key_word_from_CV

In [ ]:
NLP_Processed_CV = nlp(text)
# NLP_Processed_CV=' '.join(NLP_Processed_CV)
NLP_Processed_CV

In [ ]:
key_word = get_key_word(NLP_Processed_CV)
key_word

# Importing scapped df

In [ ]:
DF_PATH = "data.csv"

In [ ]:
df = pd.read_csv(DF_PATH)
df

In [ ]:
df["JobID"]=df.index

In [ ]:
df

In [ ]:
df.shape


In [ ]:
df.info()


In [ ]:
df['All'] = df[df.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [ ]:
df

In [ ]:
df['All']

In [ ]:
df["description"].replace(r'\n','', regex=True,inplace=True)
df["description"].replace(r'\r','', regex=True,inplace=True)  
df = df.replace(' ', '')
df.description = df.description.str.strip()
df.All = df.All.str.strip()

In [ ]:
df

# Put keywords of the CV into a dataframe

In [ ]:
df2 = pd.DataFrame()
# append columns to an empty DataFrame
df2['title'] = ["I"]
df2['job highlights'] = ["I"]
df2['job description'] = ["I"]
df2['company overview'] = ["I"]
df2['industry'] = ["I"]

# Compare with the key words from CV only
df2['All'] = " ".join(key_word)

In [ ]:
df2

In [ ]:
df.columns

# Recomendation Function

#klnsdsd

In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['positionName', 'company',"location",'JobID','description','score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        
        recommendation.at[count, 'positionName'] = df['positionName'][i]
        recommendation.at[count, 'company'] = df['company'][i]
        recommendation.at[count, 'location'] = df['location'][i]
    
        recommendation.at[count, 'JobID'] = df.index[i]
        recommendation.at[count, 'description'] = df['description'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

## cosine_similarity + TfidfVectorizer function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv)

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    output2 = list(cos_similarity_tfidf)
    return output2
    
output2 = TFIDF(df['All'], df2['All'])


In [ ]:
len(output2)

In [ ]:
output2


In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:100]
list_scores = [output2[i][0][0] for i in top]
TF=get_recommendation(top,df, list_scores)
TF

## cosine_similarity + CountVectorizer

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
count_jobid = count_vectorizer.fit_transform(df['All']) #converting job data into vectors using count vectorizers
count_jobid

In [ ]:
# print(count_jobid)


In [ ]:
user_count = count_vectorizer.transform(df2['All'])#converting user cv data into vectors using count vectorizers
user_count

In [ ]:
print(user_count)


In [ ]:
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
output3 = list(cos_similarity_countv)
#output3

In [ ]:
output3

In [ ]:
df.rating

In [ ]:
top = sorted(range(len(output3)), key=lambda i: output3[i], reverse=True)[:100]
list_scores = [output3[i][0][0] for i in top]
cv=get_recommendation(top, df, list_scores)
cv

In [ ]:
cv.description[1]

## TfidfVectorizer + NearestNeighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    
    KNN = NearestNeighbors(n_neighbors=100,p=2)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv))
    top = NNs[1][0][1:]
    index_score = NNs[0][0][1:]

    knn = get_recommendation(top, df, index_score)
    return knn

knn = KNN(df['All'], df2['All'])
knn

In [ ]:

knn.to_csv('knn.csv', index=False)

In [ ]:
cv

## Combined TFIDF, CV and KNN result together, to make a dataframe "final"


In [ ]:
merge1 = knn[['JobID','positionName', 'score']].merge(TF[['JobID','score']], on= "JobID")

In [ ]:
# merge1 = knn[['JobID','title', 'score']].merge(TF[['JobID','score']], on= "JobID")
# final = merge1.merge(cv[['JobID','score']], on = "JobID")
# final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
# final.head()a

final = merge1.merge(cv[['JobID','score']], on = "JobID")
final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
final.head()

In [ ]:
final

In [ ]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

In [ ]:
# final.to_csv('final.csv', index=False)
final2 = final.sort_values(by="Final", ascending=False).copy()
final2

In [ ]:
# final2.merge(df, on="JobID")
final_df = df.merge(final2, on="JobID")
final_df

In [ ]:
final_df.to_csv("final_df.csv",index=False)
